In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
import os
from sklearn.decomposition import PCA


In [ ]:
df = pd.read_csv('../input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
df = df[(df.drop(['date'],axis=1)>=0).all(axis=1) ] # фильтруем отрицательные значения 

In [ ]:
df.drop(['order_id' , 'line_item_type_id'], axis = 1, inplace=True)

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:

df = df[df['CPM'].between(0, df['CPM'].quantile(.95))] # убираем выбросы

In [ ]:
# убираем запрещенные переменные
df.drop(['integration_type_id' , 'revenue_share_percent','total_revenue'], axis = 1, inplace=True) 

In [ ]:

df['View/measurable'] = df.apply(lambda x: weird_division(x['viewable_impressions'],x['measurable_impressions']) , axis=1)

In [ ]:
df.drop([ 'total_impressions'], axis = 1, inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

In [ ]:
# result=[]
# for col in df.columns:
#     if '_id' in col and col not in [] :
        
#         onehot = pd.get_dummies(df[col],prefix=col[:-3])
#         result.append(onehot)
#         df.drop([col],axis=1,inplace=True)
# df =pd.concat([df]+result,axis=1)
        
        

In [ ]:
X_train = df[df['date']<pd.to_datetime('22.06.2019')]
X_test= df[df['date']>=pd.to_datetime('22.06.2019')]


y_train = X_train['CPM']
y_test = X_test['CPM']


In [ ]:
def mean_encode(X_train,X_test,columns):
    for c in columns:
        Mean_encoded_subject = X_train.groupby([c])['CPM'].mean().to_dict() 
        X_train[c] =  X_train[c].map(Mean_encoded_subject) 
        X_test[c] =  X_test[c].map(Mean_encoded_subject) 
        
    return X_train,X_test

In [ ]:
X_train,X_test = mean_encode(X_train,X_test,[c for c in X_train.columns if '_id' in c])

In [ ]:
X_train.drop(['CPM','date'],axis=1,inplace=True)
X_test.drop(['CPM','date'],axis=1,inplace=True)

In [ ]:
X_train.fillna(0,inplace=True)

In [ ]:
X_test.fillna(0,inplace=True)

# Modeling

In [ ]:
X_train

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
lr = LinearRegression()
rf=RandomForestRegressor(n_jobs=-1,n_estimators=300)
rf.fit(X_train,y_train)

In [ ]:
mean_squared_error(y_test,rf.predict(X_test))